# Baseline RAG example

This is a simple example of a baseline RAG application which purpose is to answer questions about the fantasy series [Malazan Universe](https://malazan.fandom.com/wiki/Malazan_Wiki) created by Steven Erikson and Ian C. Esslemont.

First the example will show each step of a baseline RAG pipeline including **Indexing**, **Retrieval** and **Generation**. This is done in order to show the architecture without the abstraction provided by frameworks like LlamaIndex and LangChain.
Then a more "normal" example will be shown using LlamaIndex.

As a vector database, we will use [ChromaDB](https://docs.trychroma.com/), but this can easily be exchanged with other databases.

In this example, we will use the following technologies

- OpenAI API
- ChromaDB
- LlamaIndex


### Setup libraries and environment


In [ ]:
%pip install chromadb llama-index-vector-stores-chroma

In [1]:
import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI, AzureOpenAI

import importlib
import util

#importlib.reload(util.helpers)
from util.helpers import create_and_save_md_files, get_malazan_pages, get_office_pages, get_friends_pages, get_theoffice_pages

### Environment variables

For this example you need to use an OpenAI API key. Go to [your API keys](https://platform.openai.com/api-keys) in the OpenAI console to generate one.

Then add the following to a `.env` file in the root of the project.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```


In [2]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")

In [3]:
#from dotenv import load_dotenv

#load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [5]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure",
    api_version="2024-05-01-preview"
)

chroma_client = chromadb.PersistentClient(
    path="./landsforsøg/chromadb", settings=Settings(allow_reset=True))

## Fetch documents and save them as markdown files

Here we fetch pages from the Fandom Malazan Wiki. These are the documents that we will use as our "knowledge base" in order to supply context to our prompts.

We also pre-process the content in order to be able to add them to our vector database.


In [5]:
pages = get_friends_pages()
pages
#create_and_save_md_files(pages)

[<FandomPage 'Ross Geller'>, <FandomPage 'Chandler Bing'>]

In [6]:
create_and_save_md_files(pages)

## Indexing

In this step, we will index the documents in our vector database. This will allow us to retrieve the most relevant documents when we ask a question.

We will use ChromaDB as our vector database and 'text-embedding-3-small' from OpenAI as our embedding model.


#### Fetch and process saved documents

First we need to fetch the documents we saved earlier.

Then we will process the documents in order to add them to our vector database.
The `SimpleDirectoryReader` fetches each section of the markdown file
Then each section is split in to smaller chunks of text and each chunk is embedded using the OpenAI API.


In [6]:
documents = SimpleDirectoryReader('./landsforsøg/documents').load_data()
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

document_data = []

for document in documents:
    chunks = text_splitter.split_text(document.text)
    for idx, chunk in enumerate(chunks):
        embedding = openai_client.embeddings.create(
            input=chunk, model="text-embedding-ada-002")
        document_data.append({
            "id": f"{document.id_}-{idx}",
            "text": chunk,
            "metadata": document.metadata,
            "embedding": embedding.data[0].embedding
        })

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 662 0 (offset 0)


#### Add documents to ChromaDB

In [7]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [8]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="landsforsoeg", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [9]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

## Retrieval

In this step, we will retrieve the most relevant documents to a given question. We will use the vector database to retrieve the most similar documents to the question.

In order to do this we will use the `text-embedding-3-small` model (**the same model used to index the documents**) from OpenAI to embed the question and then use the vector database to retrieve the most similar documents.

We will retrieve the top 5 documents based on the _cosine similarity_ between the question and the documents. Other similarity metrics can be used as well like squared L2 or inner product.

Change `cosine` to `l2` or `ip` when creating the collection above to try these out.


In [12]:
query = "hvordan bekæmper jeg væselhale"

In [13]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))


TABEL 5.  Bekæmpelse af væselhale i rødsvingel 2023
Rød-
svingelBehandling l pr ha. Udbytte og 
merudbytte
kg pr. ha Primo august Medio september
Mateno 
DuoBoxerMateno 
DuoBoxer 2023
Antal forsøg
 1 1.449
 2 0,35 60
 3 0,7 142
 4 0,35 11
 5 0,7 -127
 6 0,35 0,35 46
 7 0,7 1 67
 8 0,35 1 -50
 9 0,35 0,5 0,35 0,5 -34
10 0,35 1 0,35 1 -26
LSD 109TABEL 6.  Tidlig Kerb 400 SC i strandsvingel
Strand-
svingelTilført Kerb 
400 SC
l pr. haHerbicidskade forårUdbytte og merudbytte
kg pr. ha
Foder -
typePlæne -
typeBegge 
forsøgFoder -
typePlæne -
typeBegge 
forsøg
Antal forsøg 1 1 2 1 1 2
1 Ubehandlet 0 0 0,0 675 556 609
2 0,1 0,2 3 1,6 633 449 543
3 0,12 0,5 4 3,0 662 365 514
4 0,15 0,5 6 4,6 596 380 488
5 0,2 1 9 7,0 482 221 344
LSD 48 48 49

------------

61 VINTERHVEDE UKRUDT
reduktion i antal frøbærende rajgræsaks, som er resulta -
tet af sen såning og god effekt af jordmidlerne Boxer og 
Mateno Duo. Det er en vigtig erkendelse ved håndtering 
af bestande, som har ALS-resistens eller metabolisk resi -
stens.
Bekæmpelse af væselhale om efteråret
Der er udført tre forsøg i vinterhvede med bekæmpelse 
af væselhale med forskellige strategier med Boxer, Ma -
teno Duo og Atlantis OD i henholdsvis stadie 10-11 og 
stadie 12. Behandlingerne ses i tabel 18.
Forsøgene er udført på arealer med en meget stor be -
stand af væselhale, i gennemsnit ca. 600 planter pr. m2 
ved optælling i oktober. Den tidlige sprøjtning i stadie 
10-11 er udført fra 6 til 14 dage efter såning, som i gen -
nemsnit har været midt i september. Anden sprøjtning i stadie 12 er udført 9 til 13 dage senere. Der har således 
været en god timing af behandlingerne, og der har været 
gode fugtighedsforhold til at jordmidlerne har kunnet 
virke optimalt.
Effekten mod væselhale er bedømt ved optælling sidst i 
oktober og i november, dvs. henholdsvis en og to måne -
der efter behandlingen i stadie 10-11. I november er der 
også udført en visuel bedømmelse af effekten.
Forsøgsled 2 og 6 viser, at der er opnået samme effekt -
niveau af 1,5 l Boxer pr. ha og 0,7 l Mateno Duo pr. ha.

------------

ha), 330 kg pr. ha NS 27-4, og 8 kg pr.ha Absolom rød -
svingel (type med korte udløbere).
Forsøget er anlagt i et design, der gør at alle behand -
linger, udover etableringen, er de samme som den om -TABEL 10.  Bekæmpelse af væselhale i rødsvingel (J10)
RødsvingelBehandlings-
tidspunktHerbicid-
skade1)
3/4Væselhale 
pct. dækning 
af jord 3/4Væselhale 
pct. i frø ved 
høstUdb. og 
merudb., 
kg frø pr. haNettomer-
udbytte, 
kg pr. ha
2022. 2 forsøg
1.1.

------------

Erfaringer viser, at det kan være svært at håndtere udendørs, når vejret begynder at blive køligere og mere fugtigt. 
Kan en vask af klovene reducere om -
fanget af den konkrete klovsygdom?
Ved investering i en klovvasker kan det være en god idé, at vide hvilke klovlidelser, der reelt er det store problem i besætnin -
gen. Dette kan med fordel gøres ved at re -
gistrere alle klovlidelserne ved minimum en klovbeskæring. 
Den hjemmelavede ”billige” klovva -
skerløsning
Det er en mulighed at få sin lokale smed til at lave en klovvasker, som spuler bagbe -

------------

ha og 40 kg fosfor pr. ha, er der i dette forsøg ikke 
høstes et merudbytte for at placere gødningen. Forsøget 
gentages i 2023.
Rødsvingel
Bekæmpelse af væselhale i rødsvingel
I 2022 er der i samarbejde med DLF gennemført forsøg 
med at bekæmpe væselhale i rødsvingel . Der indgår to 
forsøg i denne forsøgsserie, som begge er udført i 1. års 
rødsvingel af en type med lange udløbere udlagt i vår -
byg. Væselhale skal bekæmpes så tidligt som muligt, 
hvilket vil sige kort tid efter høst af dæksæd. For at få 
bedst mulig effekt af Mateno Duo 600 SC, er det afgø -
rende, at der er en vis jordfugt til stede, hvilket kan være 
en udfordring i august måned. Som det fremgår af tabel 
10, er der ikke signifikante udbyttetab ved nogle af be -
handlingerne, og der er effekt på indholdet af væselhale 
i frøanalysen ved behandling med Mateno Duo 600 SC. 
I det ubehandlede led er der et indhold på 0,15 procent 
væselhale i frøet, og i alle behandlingerne er dette redu -
ceret. Bedste behandling er i led 3, hvor der er behand -
let med 0,7 l. Mateno Duo 600 SC pr. ha den 20. august. 
Her er der ikke fundet væselhale i frøet, og der er et lille 
merudbytte for behandlingen, merudbyttet er dog ikke 
signifikant. Som det er set i andre forsøg med Mateno 
Duo 600 SC, giver kombinationen med Boxer en bedre 
bekæmpelse af væselhale.

## Generation

In this step, we will generate an answer to the question using the retrieved documents as context. We will use the OpenAI API to generate the answer.


In [14]:
prompt = PromptTemplate("""You are a helpful assistant that answers questions about the friends characters using provided context. 

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")
message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the friends characters using provided context. 

Question: hvordan bekæmper jeg væselhale

Context: 

-----------------------------------
TABEL 5.  Bekæmpelse af væselhale i rødsvingel 2023
Rød-
svingelBehandling l pr ha. Udbytte og 
merudbytte
kg pr. ha Primo august Medio september
Mateno 
DuoBoxerMateno 
DuoBoxer 2023
Antal forsøg
 1 1.449
 2 0,35 60
 3 0,7 142
 4 0,35 11
 5 0,7 -127
 6 0,35 0,35 46
 7 0,7 1 67
 8 0,35 1 -50
 9 0,35 0,5 0,35 0,5 -34
10 0,35 1 0,35 1 -26
LSD 109TABEL 6.  Tidlig Kerb 400 SC i strandsvingel
Strand-
svingelTilført Kerb 
400 SC
l pr. haHerbicidskade forårUdbytte og merudbytte
kg pr. ha
Foder -
typePlæne -
typeBegge 
forsøgFoder -
typePlæne -
typeBegge 
forsøg
Antal forsøg 1 1 2 1 1 2
1 Ubehandlet 0 0 0,0 675 556 609
2 0,1 0,2 3 1,6 633 449 543
3 0,12 0,5 4 3,0 662 365 514
4 0,15 0,5 6 4,6 596 380 488
5 0,2 1 9 7,0 482 221 344
LSD 48 48 49

61 VINTERHVEDE UKRUDT
reduktion i antal frøbærende rajgræsaks, som er resulta -
tet af sen såning og god effekt af jordmidlerne Boxer og 
Mateno Duo. Det er en vigtig erkendelse ved håndtering 
af bestande, som har ALS-resistens eller metabolisk resi -
stens.
Bekæmpelse af væselhale om efteråret
Der er udført tre forsøg i vinterhvede med bekæmpelse 
af væselhale med forskellige strategier med Boxer, Ma -
teno Duo og Atlantis OD i henholdsvis stadie 10-11 og 
stadie 12. Behandlingerne ses i tabel 18.
Forsøgene er udført på arealer med en meget stor be -
stand af væselhale, i gennemsnit ca. 600 planter pr. m2 
ved optælling i oktober. Den tidlige sprøjtning i stadie 
10-11 er udført fra 6 til 14 dage efter såning, som i gen -
nemsnit har været midt i september. Anden sprøjtning i stadie 12 er udført 9 til 13 dage senere. Der har således 
været en god timing af behandlingerne, og der har været 
gode fugtighedsforhold til at jordmidlerne har kunnet 
virke optimalt.
Effekten mod væselhale er bedømt ved optælling sidst i 
oktober og i november, dvs. henholdsvis en og to måne -
der efter behandlingen i stadie 10-11. I november er der 
også udført en visuel bedømmelse af effekten.
Forsøgsled 2 og 6 viser, at der er opnået samme effekt -
niveau af 1,5 l Boxer pr. ha og 0,7 l Mateno Duo pr. ha.

ha), 330 kg pr. ha NS 27-4, og 8 kg pr.ha Absolom rød -
svingel (type med korte udløbere).
Forsøget er anlagt i et design, der gør at alle behand -
linger, udover etableringen, er de samme som den om -TABEL 10.  Bekæmpelse af væselhale i rødsvingel (J10)
RødsvingelBehandlings-
tidspunktHerbicid-
skade1)
3/4Væselhale 
pct. dækning 
af jord 3/4Væselhale 
pct. i frø ved 
høstUdb. og 
merudb., 
kg frø pr. haNettomer-
udbytte, 
kg pr. ha
2022. 2 forsøg
1.1.

Erfaringer viser, at det kan være svært at håndtere udendørs, når vejret begynder at blive køligere og mere fugtigt. 
Kan en vask af klovene reducere om -
fanget af den konkrete klovsygdom?
Ved investering i en klovvasker kan det være en god idé, at vide hvilke klovlidelser, der reelt er det store problem i besætnin -
gen. Dette kan med fordel gøres ved at re -
gistrere alle klovlidelserne ved minimum en klovbeskæring. 
Den hjemmelavede ”billige” klovva -
skerløsning
Det er en mulighed at få sin lokale smed til at lave en klovvasker, som spuler bagbe -

ha og 40 kg fosfor pr. ha, er der i dette forsøg ikke 
høstes et merudbytte for at placere gødningen. Forsøget 
gentages i 2023.
Rødsvingel
Bekæmpelse af væselhale i rødsvingel
I 2022 er der i samarbejde med DLF gennemført forsøg 
med at bekæmpe væselhale i rødsvingel . Der indgår to 
forsøg i denne forsøgsserie, som begge er udført i 1. års 
rødsvingel af en type med lange udløbere udlagt i vår -
byg. Væselhale skal bekæmpes så tidligt som muligt, 
hvilket vil sige kort tid efter høst af dæksæd. For at få 
bedst mulig effekt af Mateno Duo 600 SC, er det afgø -
rende, at der er en vis jordfugt til stede, hvilket kan være 
en udfordring i august måned. Som det fremgår af tabel 
10, er der ikke signifikante udbyttetab ved nogle af be -
handlingerne, og der er effekt på indholdet af væselhale 
i frøanalysen ved behandling med Mateno Duo 600 SC. 
I det ubehandlede led er der et indhold på 0,15 procent 
væselhale i frøet, og i alle behandlingerne er dette redu -
ceret. Bedste behandling er i led 3, hvor der er behand -
let med 0,7 l. Mateno Duo 600 SC pr. ha den 20. august. 
Her er der ikke fundet væselhale i frøet, og der er et lille 
merudbytte for behandlingen, merudbyttet er dog ikke 
signifikant. Som det er set i andre forsøg med Mateno 
Duo 600 SC, giver kombinationen med Boxer en bedre 
bekæmpelse af væselhale.

-----------------------------------



In [15]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Væselhale er en type insekt, der typisk trives i fugtige områder. Bekæmpelsen kan kræve flere metoder. 

1. Fjern fugt: Den første og vigtigste metode er at fjerne fugtkilder, da væselhale trives i fugtige omgivelser. Tjek for lækager i rør og reparer dem. Installer et affugtningssystem, hvis fugtigheden i dit hjem er høj.

2. Rengøring: Rengør dit hjem grundigt, især mørke og fugtige områder som badeværelser, kældre og lofter. Væselhale klækker fra æg, der kan gemme sig i små sprækker og revner.

3. Professionelle Insektmidler: Hvis problemet vedvarer, skal du overveje at bruge et insektmiddel, der er specielt designet til at bekæmpe væselhale. Følg alle sikkerhedsanvisningerne på etiketten.

4. Brug af kiselgur: Dette er en naturlig substans, der kan hjælpe med at bekæmpe væselhale. Det er et fint pulver, der kan strøs omkring områder, hvor du har bemærket aktivitet.

5. Professionel Bistand: Hvis væselhaleinfestationen ikke forsvinder på trods af dine bestræbelser, kan det være nødvendigt at ansætte en professionel skadedyrsbekæmper.

Husk, det er nemmere at forebygge en infestation end at bekæmpe den. Hold dit hjem rent og tørt, og vær opmærksom på mulige tegn på infestation, såsom at se væselhale i hjemmet.

## Normal example using LlamaIndex

In this example, we will use LlamaIndex to abstract the indexing and retrieval steps. This shows how easily the same pipeline can be implemented using LlamaIndex.


In [ ]:
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai

In [88]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

#from openai import OpenAI, AzureOpenAI

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="landsforsoeg", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embedding,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [85]:
# loading from saved
collection = chroma_client.get_or_create_collection(
    name="friends", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embedding)

In [89]:
# Fetch documents
documents = SimpleDirectoryReader('./landsforsøg/documents').load_data()

# Run pipeline
pipeline.run(documents=documents) # num_workers=4

print("Done")

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 662 0 (offset 0)


Done


#### Create base QueryEngine from LlamaIndex


In [25]:
query_engine = index.as_query_engine(llm=llm, verbose=True)

#### Or alternatively, create a CustomQueryEngine


In [86]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer

from llama_index.core.postprocessor import TimeWeightedPostprocessor

qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about landsforsøgene using provided context.
    Answers must be provided in the Danish language and in the form of a list if multiple steps or points are included in the answer.
    Below the answer, relevant metadata, i.e. filenames and page numbers should be provided.
    Example: Kilde: filename.pdf, side 60-62.

    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate
    postprocessor: TimeWeightedPostprocessor

    def custom_query(self, query_str: str):
        nodes = self.retriever.retrieve(query_str)
        context_str = "\n\n".join([n.node.get_content(metadata_mode="all") for n in nodes])
        #context = qa_prompt.format(
        #    context_str=context_str, query_str=query_str)
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )
                    
        return str(response) + "\n\n-------------------------\n\nKontekst:\n\n" + context_str


synthesizer = get_response_synthesizer(response_mode="compact")

node_postprocessor = TimeWeightedPostprocessor(
    time_decay=0.5, time_access_refresh=False, top_k=1
)

query_engine = RAGQueryEngine(
    retriever=index.as_retriever(), #similarity_top_k=2
    response_synthesizer=synthesizer,
    postprocessor = node_postprocessor,
    llm=llm,
    qa_prompt=qa_prompt,
)

In [ ]:
query = "hvordan bekæmper jeg væselhale?"
response = query_engine.query(query)
display(Markdown(f"{response}"))

## Simplest RAG implementation using LlamaIndex


In [19]:

from llama_index.core import Settings

llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

Settings.llm = llm
Settings.embed_model = embedding


# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents)

# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

# Use your Default RAG
response = query_engine.query(query)
display(Markdown(f"{response}"))

Chandler starts off as a Data Processor, a job he dislikes. He later becomes a Processing Supervisor. His work involves "Statistical Analysis and Data Reconfiguration," which is revealed when he quits his job in season 9. After quitting his job, he briefly runs a new branch of the company in Tulsa, Oklahoma, but eventually quits that too. Monica then helps him secure a job in advertising, starting as an unpaid intern and later becoming a junior copywriter. In an alternate reality storyline, Chandler quits his job and works as a freelance writer, specializing in comics.

In [20]:
query = "how many women did Ross date and what were their names?"
response = query_engine.query(query)
display(Markdown(f"{response}"))

Ross dated three women mentioned in the context: Rachel, Emily Waltham, and Elizabeth Stevens. The context also mentions that Ross slept with 14 women during the series, but their names are not provided.